In [1]:
import pandas as pd, numpy as np, random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# !pip install tensorflow==2.0.0-alpha0
import tensorflow as tf
from tensorflow import keras
tf.__version__,keras.__version__,tf.test.is_gpu_available()

('2.0.0-alpha0', '2.2.4-tf', False)

# Test Integer Sequences

https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/

- Input: sequence of 5 integers
- Output: the first two integers, then padded with 0; or other criterions

In [218]:
seqlen = 5
maxint = 60
batchsize = 32

## Generate Data

In [260]:
one_hot_decode = lambda X: np.argmax(X,axis=X.ndim-1)
 
def gen_pairs(batchsize=2, onehot=True):
    while True:
        X = np.random.randint(0, maxint-1,(batchsize,seqlen),dtype=int)
        y = X.copy(); 
#         y = np.hstack((y[:,:2],np.zeros((batchsize,3),dtype=int)))  # original: first 2 integers
#         y = np.clip(y,maxint//3,2*maxint//3)
#         y.sort(axis=1)
        y = np.cumsum(y,1)/np.tile(y.sum(1),(seqlen,1)).T*(maxint-1); y = y.astype(int)
        
        if onehot: #https://stackoverflow.com/a/49217762
            I = np.eye(maxint)
            X = I[X.reshape(-1)].reshape(*X.shape,maxint)
            y = I[y.reshape(-1)].reshape(*y.shape,maxint)
        
        yield X,y

X,y = next(gen_pairs(2,False))
X,y

(array([[ 3, 57, 19,  5, 28],
        [47, 31, 34, 21, 54]]), array([[ 1, 31, 41, 44, 59],
        [14, 24, 35, 41, 59]]))

## Keras Model w/o Attention

In [264]:
# define 
cellSize = 100
model = tf.keras.models.Sequential()
# model.add(keras.layers.LSTM(cellSize, input_shape=(seqlen, maxint)))  #original
# model.add(keras.layers.RepeatVector(seqlen))                          #original
# model.add(keras.layers.LSTM(cellSize, return_sequences=True))         #original
model.add(keras.layers.Bidirectional(keras.layers.LSTM(cellSize, return_sequences=True, input_shape=(seqlen, maxint))))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(cellSize, return_sequences=True)))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(maxint,
                                                          activation='softmax')
                                      ))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [266]:
# Train
import tempfile
_,weightsfile=tempfile.mkstemp()
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
es=EarlyStopping(monitor='val_acc', patience=3, verbose=1)
mc=ModelCheckpoint(weightsfile,monitor='val_acc',save_best_only=True,save_weights_only=True,verbose=0)

model.fit_generator(gen_pairs(32, True), steps_per_epoch=1000, epochs=1000,callbacks=[es,mc],
                    validation_data=gen_pairs(32, True),validation_steps=100)
model.load_weights(weightsfile)

Epoch 1/1000
1000/1000 [==============================] - 12s 12ms/step - loss: 0.2968 - acc: 0.8760 - val_loss: 0.3174 - val_acc: 0.8634
Epoch 2/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.2891 - acc: 0.8790 - val_loss: 0.3023 - val_acc: 0.8734
Epoch 3/1000
1000/1000 [==============================] - 13s 13ms/step - loss: 0.2782 - acc: 0.8834 - val_loss: 0.2819 - val_acc: 0.8812
Epoch 4/1000
1000/1000 [==============================] - 12s 12ms/step - loss: 0.2772 - acc: 0.8832 - val_loss: 0.2454 - val_acc: 0.8971
Epoch 5/1000
1000/1000 [==============================] - 12s 12ms/step - loss: 0.2733 - acc: 0.8848 - val_loss: 0.2796 - val_acc: 0.8813
Epoch 6/1000
1000/1000 [==============================] - 12s 12ms/step - loss: 0.2710 - acc: 0.8860 - val_loss: 0.2937 - val_acc: 0.8768
Epoch 7/1000
1000/1000 [==============================] - 12s 12ms/step - loss: 0.2604 - acc: 0.8909 - val_loss: 0.2790 - val_acc: 0.8801
Epoch 00007: early stopping


In [267]:
# Evaluate
X,y = next(gen_pairs(10000,True))
yhat = model.predict(X, verbose=0)
wrong = list(map(np.any, one_hot_decode(y)-one_hot_decode(yhat))) #np.any returns true (1) if any element is nonzero
print(f'Accuracy: {1-sum(wrong)/y.shape[0]}%') # different from acc during training as we count the WHOLE sequence as right/wrong here

for i in np.where(wrong)[0][:10]:   # output examples that are wrong
    print(f'True:{one_hot_decode(y[i])}, Prediction:{one_hot_decode(yhat[i])}')

Accuracy: 0.5846%
True:[ 0  4 20 43 59], Prediction:[ 0  5 21 43 59]
True:[ 2 11 30 58 59], Prediction:[ 2 12 30 58 59]
True:[11 31 37 49 59], Prediction:[10 31 38 49 59]
True:[21 22 39 42 59], Prediction:[20 22 39 43 59]
True:[14 25 40 52 59], Prediction:[14 25 40 51 59]
True:[23 37 38 53 59], Prediction:[23 36 37 53 59]
True:[10 23 49 50 59], Prediction:[10 22 50 50 59]
True:[12 19 35 53 59], Prediction:[12 18 35 53 59]
True:[ 5 13 35 53 59], Prediction:[ 4 13 35 53 59]
True:[ 2 29 29 48 59], Prediction:[ 2 29 29 49 59]


## Keras Model w/ Attention

In [2]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent, _time_distributed_dense
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

from keras.models import Sequential
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(150, input_shape=(n_timesteps_in, n_features), return_sequences=True))
model.add(AttentionDecoder(150, n_features))
model.summary()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

# Translate dates to standard format

https://www.coursera.org/learn/nlp-sequence-models/notebook/npjGi/neural-machine-translation-with-attention

# language translation 

https://www.tensorflow.org/alpha/tutorials/text/nmt_with_attention

In [4]:
import urllib, os, tempfile, zipfile
os.chdir(tempfile.gettempdir())
urllib.request.urlretrieve('http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip','spa-eng.zip')
zipfile.ZipFile('spa-eng.zip').extractall()
# ./spa-eng/spa.txt

In [11]:
import unicodedata, re

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # "he is a boy." => "he is a boy ." (https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation)
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w

print(preprocess_sentence("May I borrow this book?"))
print(preprocess_sentence("¿Puedo tomar prestado este libro?").encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'
